In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=427ec71bbfe8ea12497228406c3dfbd3912593b7a28a5ce0f9497e399a8fd2ae
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [5]:
!pip install translate

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, when, regexp_replace, regexp_extract, initcap, concat_ws, length, col, lit, udf
import pyspark.sql.functions as F
import os
from time import sleep
from pyspark.sql.types import StringType
from translate import Translator


In [7]:
os.chdir('/content/drive/MyDrive/Bol_Data_Processing')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"/content/drive/MyDrive/Bol_Data_Processing/google_keys.json"
current_dir = os.getcwd()
current_dir

'/content/drive/MyDrive/Bol_Data_Processing'

In [8]:
spark = SparkSession.builder.getOrCreate()
print(spark)

## Read CSV and to CSV using spark

In [9]:
def read_csv(dir):
    data = spark.read \
        .format('csv') \
        .option('header', 'true') \
        .load(current_dir + dir)
    return data

In [10]:
def to_csv(data, dir):
    data.to_csv(current_dir + dir, index=False)

## Computers & Accesories

### Category

In [11]:
categories = read_csv('/Data crawled/category_computers.csv')

In [12]:
categories.show(truncate=False)

+---------+-----------+
|category |category_id|
+---------+-----------+
|Laptops  |01         |
|Monitoren|02         |
|Desktops |03         |
|Beamers  |04         |
|Software |05         |
+---------+-----------+



Translating the `category` column to English

In [13]:
translator = Translator(to_lang='en', from_lang='nl')


In [14]:
def translate_text(text):
    if text is not None:
        translated = translator.translate(text)
        return translated
    else:
        return None

In [15]:
translate_udf = udf(translate_text)

In [16]:
categories = categories.withColumn("translated_categories", translate_udf("category"))
categories.show(truncate=False)

+---------+-----------+---------------------+
|category |category_id|translated_categories|
+---------+-----------+---------------------+
|Laptops  |01         |Laptops              |
|Monitoren|02         |Monitoring           |
|Desktops |03         |Desktops             |
|Beamers  |04         |Beamers              |
|Software |05         |Software             |
+---------+-----------+---------------------+



### Brands

In [17]:
brands = read_csv('/Data crawled/brands_computers_accessories.csv')

In [18]:
brands.show(truncate=False)

+----------------+---------+-----------+
|brands          |brands_id|category_id|
+----------------+---------+-----------+
|Acer            |01       |1          |
|Acer Computers  |02       |1          |
|Acer LiteON     |03       |1          |
|Alienware       |04       |1          |
|Alienware 17 R2 |05       |1          |
|Apple           |06       |1          |
|ASU             |07       |1          |
|ASUS            |08       |1          |
|Asus rog        |09       |1          |
|Asus Rog Strix  |10       |1          |
|Avita           |11       |1          |
|Blauw           |12       |1          |
|BTO             |13       |1          |
|Clevo           |14       |1          |
|Concept         |15       |1          |
|ConceptD        |16       |1          |
|Dell            |17       |1          |
|Dynabook        |18       |1          |
|Dynabook Toshiba|19       |1          |
|ElementKey      |20       |1          |
+----------------+---------+-----------+
only showing top

Transform the brand names by capitalizing the first letter

In [20]:
brands = brands.withColumn('brands', initcap('brands'))
brands.toPandas()

,brands,brands_id,category_id
0,Acer,01,1
1,Acer Computers,02,1
2,Acer Liteon,03,1
3,Alienware,04,1
4,Alienware 17 R2,05,1
...,...,...,...
440,Vipre,441,5
441,Westra On Bridge,442,5
442,Windows,443,5
443,Woord Voor Woord,444,5


### Products

In [21]:
products = read_csv('/Data crawled/computers_accesories.csv')
products_mod = products.withColumnRenamed("phone_specs", "product_specs")
products_mod = products.withColumn("product_specs", products["phone_specs"]).drop("phone_specs")
products_mod.toPandas()

,product_name,prices,ratings,short_descs,other_options,category_id,brand_id,product_specs
0,HP 15s-fq2730nd - Laptop - 15.6 inch,399.00,"4,7 van de 5 sterren uit 3",Deze HP 15s-fq2730nd laptop is geschikt voor d...,None,1,26,15.6 Inch Full HD Intel Core i3 processor RAM:...
1,HP 14s-fq1716nd - Laptop - 14 inch,519.00,"4,2 van de 5 sterren uit 6",Deze HP 14s-fq1716nd laptop is geschikt voor h...,None,1,26,14 Inch Full HD AMD Ryzen 5 processor RAM: 16 ...
2,HP 15s-fq5750nd - Laptop - 15.6 inch,None,None,None,None,1,26,15.6 Inch Full HD Intel Core i5 processor RAM:...
3,HP 15s-fq2771nd - Laptop - 15.6 inch,699.00,"4,7 van de 5 sterren uit 3",Deze HP 15s-fq2771nd laptop is geschikt voor h...,None,1,26,15.6 Inch Full HD Intel Core i7 processor RAM:...
4,Lenovo IdeaPad 3 14ADA6 82KQ000SMH - Laptop - ...,349.00,"4,7 van de 5 sterren uit 6",Uitvoeren van basis taken met weinig opslag en...,None,1,31,14 Inch Full HD AMD Athlon Silver processor RA...
...,...,...,...,...,...,...,...,...
11598,Planten /PC,None,None,None,None,5,427.0,None
11599,F-Secure Safe 3-Devices 1 jaar,None,None,None,None,5,383.0,3 Platform: Windows
11600,F-Secure Safe 3-Devices 2 jaar,None,None,None,None,5,383.0,"3 Platform: Windows, Android, iOS"
11601,Digikidz Werkwoordenrom / Groep 5/6,None,None,None,None,5,355.0,Platform: Windows


Delete the "... Meer" at the end of `short_desc`

In [304]:
products_mod = products_mod.withColumn("short_descs", F.trim(F.expr("substring(short_descs, 1, length(short_descs) - length('... Meer'))")))
products_mod.show(truncate=False)

,product_name,prices,ratings,short_descs,other_options,category_id,brand_id,product_specs
0,HP 15s-fq2730nd - Laptop - 15.6 inch,399.00,"4,7 van de 5 sterren uit 3",Deze HP 15s-fq2730nd laptop is geschikt voor d...,None,1,26,15.6 Inch Full HD Intel Core i3 processor RAM:...
1,HP 14s-fq1716nd - Laptop - 14 inch,519.00,"4,2 van de 5 sterren uit 6",Deze HP 14s-fq1716nd laptop is geschikt voor h...,None,1,26,14 Inch Full HD AMD Ryzen 5 processor RAM: 16 ...
2,HP 15s-fq5750nd - Laptop - 15.6 inch,None,None,None,None,1,26,15.6 Inch Full HD Intel Core i5 processor RAM:...
3,HP 15s-fq2771nd - Laptop - 15.6 inch,699.00,"4,7 van de 5 sterren uit 3",Deze HP 15s-fq2771nd laptop is geschikt voor h...,None,1,26,15.6 Inch Full HD Intel Core i7 processor RAM:...
4,Lenovo IdeaPad 3 14ADA6 82KQ000SMH - Laptop - ...,349.00,"4,7 van de 5 sterren uit 6",Uitvoeren van basis taken met weinig opslag en...,None,1,31,14 Inch Full HD AMD Athlon Silver processor RA...
...,...,...,...,...,...,...,...,...
11598,Planten /PC,None,None,None,None,5,427.0,None
11599,F-Secure Safe 3-Devices 1 jaar,None,None,None,None,5,383.0,3 Platform: Windows
11600,F-Secure Safe 3-Devices 2 jaar,None,None,None,None,5,383.0,"3 Platform: Windows, Android, iOS"
11601,Digikidz Werkwoordenrom / Groep 5/6,None,None,None,None,5,355.0,Platform: Windows


Translate the `ratings`, `short_desc`, `product_specs` to English

In [306]:
columns_to_translate = ["ratings", "short_descs", "product_specs "]

translated_products = products_mod.select(
    *[translate_udf(col(column)).alias(column) if column in columns_to_translate else col(column) for column in products_mod.columns]
)
translated_products = translated_products.na.fill("")
translated_products.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------+-------+----------------------------+-----------------------------------------------------------------------------------------------------------+-------------+-----------+--------+-----------------------------------------------------------------------------------------------------------------------------+
|product_name                                                                                                                                  |prices |ratings                     |short_descs                                                                                                |other_options|category_id|brand_id|product_specs                                                                                                                |
+-------------------------------------------------------------------------------------------------